In [1]:
!git clone https://github.com/jjordanoc/CUT3R.git /root/jupyter/CUT3R
%cd /root/jupyter/CUT3R

fatal: destination path '/root/jupyter/CUT3R' already exists and is not an empty directory.
/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/CUT3R


/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# Install PyTorch with CUDA support (Colab already has CUDA 12.x)
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121

# Install requirements
!pip install -r requirements.txt

# Install additional dependencies
!pip install git+https://github.com/nerfstudio-project/gsplat.git
!pip install evo open3d


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 170.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 162.2 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for filelock from https://download.pytorch.org/whl/filelock-3.19.1-py3-none-any.whl.metadata
  Obtaining dependency information for networkx from https://download.pytorch.org/whl/networkx-3.5-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec from https://download.pytorch.org/whl/fsspec-2025.9.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 165.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 245.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 196.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 137.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
%ls

LICENSE           assets/               demo.py     examples/
README.md         cloud_opt/            demo_ga.py  requirements.txt
__pycache__/      config/               docs/       src/
add_ckpt_path.py  datasets_preprocess/  eval/       viser_utils.py


In [4]:
# Navigate to the CUT3R directory
#%cd /CUT3R
# Apply the fix using sed
!sed -i 's/tokens\.type()/tokens.scalar_type()/g' src/croco/models/curope/kernels.cu

# Rebuild the extension
%cd src/croco/models/curope/
!python setup.py build_ext --inplace


/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/CUT3R/src/croco/models/curope
running build_ext
/usr/local/lib/python3.10/site-packages/torch/utils/cpp_extension.py:497: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/site-packages/torch/utils/cpp_extension.py:426: UserWarning: There are no clang++ version bounds defined for CUDA version 12.1
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
building 'curope' extension
clang -pthread -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -I/tools/deps/include -I/tools/deps/include/ncursesw -I/tools/deps/libedit/include -g0 -fPIC -I/usr/local/lib/python3.10/site-packages/torch/include -I/usr/local/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -I/usr/local

In [6]:
import os
if not os.path.exists("/root/jupyter/CUT3R/src/cut3r_512_dpt_4_64.pth"):
    %cd /root/jupyter/CUT3R/src
    !pip install gdown
    !gdown --fuzzy https://drive.google.com/file/d/1Asz-ZB3FfpzZYwunhQvNPZEUA8XUNAYD/view?usp=drive_link

In [14]:
import os
from PIL import Image
from pathlib import Path
import shutil

src_dirs = [
    # "/root/jupyter/data/iphone13",
    "/root/jupyter/data/iphone6",
    # "/root/jupyter/data/samsung-s35-5g",
]
dst_dir = Path("/root/jupyter/data/resized_classified/iphone6/landscape")
dst_dir.mkdir(parents=True, exist_ok=True)

def resize_to_max512(img: Image.Image) -> Image.Image:
    w, h = img.size
    if max(w, h) <= 512:
        return img  # no resize needed
    scale = 512 / max(w, h)
    new_size = (int(w * scale), int(h * scale))
    return img.resize(new_size, Image.LANCZOS)

for src in src_dirs:
    for path in Path(src).rglob("*"):
        if path.suffix.lower() in {".jpg", ".jpeg", ".png"}:
            try:
                img = Image.open(path)
                # only landscape images
                if img.width > img.height:
                    img = resize_to_max512(img)
                    # keep filename but flatten structure
                    out_path = dst_dir / path.name
                    img.save(out_path)
                    img.close()
                    print(f"✅ Saved {out_path}")
            except Exception as e:
                print(f"❌ Skipped {path}: {e}")

✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0041.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0042.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0043.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0044.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0045.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0046.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0047.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0048.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0049.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0050.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0051.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0052.JPG
✅ Saved /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0053.JPG

In [10]:
!apt-get update && apt-get install -y libgomp1

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3755 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [5936 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3428 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5735 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2080 kB]
Fetched 21.3 MB in 2s (13.3 MB/s)                          
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state informa

In [12]:
import torch
imgs = torch.load("/root/jupyter/data/preprocessed_data.pth", weights_only=False)

OSError: libgomp-a34b3233.so.1: cannot open shared object file: No such file or directory

In [15]:
from datetime import datetime

# Generate human-readable timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_dir = f"/root/jupyter/reconstruction/{timestamp}"

seq_path = "/root/jupyter/data/resized_classified/iphone6/landscape"



%cd /root/jupyter/CUT3R/
!python demo_ga.py --model_path src/cut3r_512_dpt_4_64.pth --size 512 --seq_path {seq_path} --vis_threshold 1.5 --output_dir {output_dir}

/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/CUT3R
Found 242 images in /root/jupyter/data/resized_classified/iphone6/landscape.
Preparing input views...
>> Loading a list of 242 images
 - adding /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0041.JPG with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0042.JPG with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0043.JPG with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0044.JPG with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0045.JPG with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0046.JPG with resolution 512x384 --> 512x384
 - adding /root/jupyter/data/resized_classified/iphone6/landscape/IMG_0047.JPG with resolution 512x384 --> 512x384
 - adding /r

In [ ]:
!python demo.py --model_path src/cut3r_512_dpt_4_64.pth --size 512 --seq_path data/video.mp4 --vis_threshold 1.5 --output_dir reconstruction

2025-10-11 01:54:35.973177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760147676.216456   11519 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760147676.282184   11519 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760147676.777319   11519 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760147676.777358   11519 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760147676.777362   11519 computation_placer.cc:177] computation placer alr

In [ ]:
#!cat demo_ga.py
#!git pull

#!/usr/bin/env python3
"""
3D Point Cloud Inference and Visualization Script

This script performs inference using the ARCroco3DStereo model and visualizes the
resulting 3D point clouds with the PointCloudViewer. Use the command-line arguments
to adjust parameters such as the model checkpoint path, image sequence directory,
image size, device, etc.

Usage:
    python demo_ga.py [--model_path MODEL_PATH] [--seq_path SEQ_PATH] [--size IMG_SIZE]
                            [--device DEVICE] [--vis_threshold VIS_THRESHOLD] [--output_dir OUT_DIR]

Example:
    python demo_ga.py --model_path src/cut3r_512_dpt_4_64.pth \
        --seq_path examples/001 --device cuda --size 512
"""

import os
import numpy as np
import torch
import time
import glob
import random
import cv2
import argparse
import tempfile
import shutil
from copy import deepcopy
from add_ckpt_path import add_path_to_dust3r
import imageio.v2 as iio

# Set random seed for reproducibility.
random.seed(42)

def forward_backward_permu